# Tucker Tensors

```
Copyright 2022 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

Tucker format is a decomposition of a tensor X as the product of a core tensor G and matrices (e.g., A,B,C) in each dimension. In other words, a tensor X is expressed as:

$
\mathcal{X} = \mathcal{G} \times_1 A \times_2 B \times_3 C
$

In MATLAB notation: `X=ttm(G,{A,B,C})`

## Contents

* [Creating a ttensor with a tensor core](#Creating-a-ttensor-with-a-tensor-core)
* [Alternate core formats: sptensor, ktensor, or ttensor](#Alternate-core-formats:-sptensor,-ktensor,-or-ttensor)
* [Creating a one-dimensional ttensor](#Creating-a-one-dimensional-ttensor)
* [Constituent parts of a ttensor](#Constituent-parts-of-a-ttensor)
* [Creating a ttensor from its constituent parts](#Creating-a-ttensor-from-its-constituent-parts)
* [Creating an empty ttensor](#Creating-an-empty-ttensor)
* [Use full or tensor to convert a ttensor to a tensor](#Use-full-or-tensor-to-convert-a-ttensor-to-a-tensor)
* [Use reconstruct to compute part of a full tensor](#Use-reconstruct-to-compute-part-of-a-full-tensor)
* [Use double to convert a ttensor to a (multidimensional) array](#Use-double-to-convert-a-ttensor-to-a-(multidimensional)-array)
* [Use ndims and size to get the size of a ttensor](#Use-ndims-and-size-to-get-the-size-of-a-ttensor)
* [Subscripted reference for a ttensor](#Subscripted-reference-for-a-ttensor)
* [Subscripted assignment for a ttensor](#Subscripted-assignment-for-a-ttensor)
* [Last index](#Last-index)
* [Basic operations (uplus, uminus, mtimes, etc.) on a ttensor](#Basic-operations-(uplus,-uminus,-mtimes,-etc.)-on-a-ttensor)
     * [Addition](#Addition)
     * [Subtraction](#Subtraction)
     * [Multiplication](#Multiplication)
* [Use permute to reorder the modes of a ttensor](#Use-permute-to-reorder-the-modes-of-a-ttensor)

In [1]:
import pyttb as ttb
import numpy as np

In [2]:
# Upcoming ttensors will be generated with this same initialization. 
def generate_sample_ttensor() -> ttb.ttensor:
    np.random.seed(0)
    core = ttb.tensor.from_data(np.random.rand(3,2,1), shape=(3,2,1)) # The core tensor.
    U = [np.random.rand(5,3), np.random.rand(4,2), np.random.rand(3,1)] # The factor matrices.
    X = ttb.ttensor.from_data(core, U) # Create the ttensor.
    
    return X

## Creating a ttensor with a tensor core

In [3]:
X = generate_sample_ttensor() # Create the ttensor.
X

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[0.5488135 ]
	 [0.71518937]]
	data[1, :, :] = 
	[[0.60276338]
	 [0.54488318]]
	data[2, :, :] = 
	[[0.4236548 ]
	 [0.64589411]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]

## Alternate core formats: sptensor, ktensor, or ttensor

## **TODO**: "support any tensor type with supported ops"

In [4]:
np.random.seed(0)
# sptensor_core = ttb.sptenrand([3,2,1], nonzeros=3) # Create a 3 x 2 x 1 sptensor.
# U = [np.random.rand(5,3), np.random.rand(4,2), np.random.rand(3,1)] # The factor matrices.
# Y = ttb.ttensor.from_data(sptensor_core, U) # # Core is a sptensor.

In [5]:
V = [np.random.rand(3,2), np.random.rand(2,2), np.random.rand(1,2)] # Create some random matrices.
# ktensor_core = ttb.ktensor.from_factor_matrices(V) # Create a 3 x 2 x 1  ktensor.
# Y = ttb.ttensor.from_data(ktensor_core, V) # Core is a ktensor.

In [6]:
dense_tensor = ttb.tensor.from_data(np.random.rand(3,2,1), (3,2,1)) # Core is a tensor.
U = [np.random.rand(5,3), np.random.rand(4,2), np.random.rand(3,1)] # The factor matrices.
Y = ttb.ttensor.from_data(dense_tensor, U) # Create the ttensor.
Y

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[0.56804456]
	 [0.92559664]]
	data[1, :, :] = 
	[[0.07103606]
	 [0.0871293 ]]
	data[2, :, :] = 
	[[0.0202184 ]
	 [0.83261985]]
	U[0] = 
		[[0.77815675 0.87001215 0.97861834]
		 [0.79915856 0.46147936 0.78052918]
		 [0.11827443 0.63992102 0.14335329]
		 [0.94466892 0.52184832 0.41466194]
		 [0.26455561 0.77423369 0.45615033]]
	U[1] = 
		[[0.56843395 0.0187898 ]
		 [0.6176355  0.61209572]
		 [0.616934   0.94374808]
		 [0.6818203  0.3595079 ]]
	U[2] = 
		[[0.43703195]
		 [0.6976312 ]
		 [0.06022547]]

## Creating a one-dimensional ttensor

In [7]:
np.random.seed(0)
dense_tensor = ttb.tensor.from_data(2*np.random.rand(2,1), (2,)) # Core tensor.
Z = ttb.ttensor.from_data(dense_tensor, [np.random.rand(4,2)]) # One-dimensional ttensor.
Z

Tensor of shape: (4,)
	Core is a 	tensor of shape 2
	data[:] = 
	[1.09762701 1.43037873]
	U[0] = 
		[[0.60276338 0.54488318]
		 [0.4236548  0.64589411]
		 [0.43758721 0.891773  ]
		 [0.96366276 0.38344152]]

## Constituent parts of a ttensor

In [8]:
X = generate_sample_ttensor() # Create the ttensor.
X.core # Core tensor.

tensor of shape 3 x 2 x 1
data[0, :, :] = 
[[0.5488135 ]
 [0.71518937]]
data[1, :, :] = 
[[0.60276338]
 [0.54488318]]
data[2, :, :] = 
[[0.4236548 ]
 [0.64589411]]

In [9]:
X.u # Cell array of matrices.

[array([[0.43758721, 0.891773  , 0.96366276],
        [0.38344152, 0.79172504, 0.52889492],
        [0.56804456, 0.92559664, 0.07103606],
        [0.0871293 , 0.0202184 , 0.83261985],
        [0.77815675, 0.87001215, 0.97861834]]),
 array([[0.79915856, 0.46147936],
        [0.78052918, 0.11827443],
        [0.63992102, 0.14335329],
        [0.94466892, 0.52184832]]),
 array([[0.41466194],
        [0.26455561],
        [0.77423369]])]

## Creating a ttensor from its constituent parts

In [10]:
X = generate_sample_ttensor() # Create the ttensor.
Y = ttb.ttensor.from_data(X.core, X.u) # Recreate a ttensor from its parts.
Y

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[0.5488135 ]
	 [0.71518937]]
	data[1, :, :] = 
	[[0.60276338]
	 [0.54488318]]
	data[2, :, :] = 
	[[0.4236548 ]
	 [0.64589411]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]

## Creating an empty ttensor

In [11]:
X = ttb.ttensor() # Empty ttensor.
X

Tensor of shape: ()
	Core is a 	empty tensor of shape ()
	data = []

## Use full or tensor to convert a ttensor to a tensor

In [12]:
X = generate_sample_ttensor() # Create a ttensor.
X

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[0.5488135 ]
	 [0.71518937]]
	data[1, :, :] = 
	[[0.60276338]
	 [0.54488318]]
	data[2, :, :] = 
	[[0.4236548 ]
	 [0.64589411]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]

In [13]:
X.full() # Converts to a tensor.

tensor of shape 5 x 4 x 3
data[0, :, :] = 
[[0.66497416 0.42425559 1.24160273]
 [0.45354281 0.28936173 0.84682989]
 [0.39917702 0.25467618 0.74532111]
 [0.77210991 0.49260853 1.44164064]]
data[1, :, :] = 
[[0.50252742 0.32061407 0.93829123]
 [0.34644731 0.22103447 0.64686714]
 [0.30417958 0.19406752 0.56794717]
 [0.58375429 0.37243706 1.08995351]]
data[2, :, :] = 
[[0.48119404 0.3070033  0.89845873]
 [0.3381225  0.21572321 0.63132351]
 [0.29560878 0.18859932 0.55194425]
 [0.55942684 0.35691607 1.04453065]]
data[3, :, :] = 
[[0.25371838 0.16187312 0.47372884]
 [0.16355973 0.10435162 0.30538962]
 [0.14584965 0.09305254 0.27232236]
 [0.29392047 0.18752218 0.54879195]]
data[4, :, :] = 
[[0.77085415 0.49180735 1.43929596]
 [0.52368038 0.33410972 0.97778686]
 [0.46132233 0.29432509 0.86135538]
 [0.89490068 0.57094943 1.67090873]]

In [14]:
ttb.tensor.from_tensor_type(X) # Also converts to a tensor.

tensor of shape 5 x 4 x 3
data[0, :, :] = 
[[0.66497416 0.42425559 1.24160273]
 [0.45354281 0.28936173 0.84682989]
 [0.39917702 0.25467618 0.74532111]
 [0.77210991 0.49260853 1.44164064]]
data[1, :, :] = 
[[0.50252742 0.32061407 0.93829123]
 [0.34644731 0.22103447 0.64686714]
 [0.30417958 0.19406752 0.56794717]
 [0.58375429 0.37243706 1.08995351]]
data[2, :, :] = 
[[0.48119404 0.3070033  0.89845873]
 [0.3381225  0.21572321 0.63132351]
 [0.29560878 0.18859932 0.55194425]
 [0.55942684 0.35691607 1.04453065]]
data[3, :, :] = 
[[0.25371838 0.16187312 0.47372884]
 [0.16355973 0.10435162 0.30538962]
 [0.14584965 0.09305254 0.27232236]
 [0.29392047 0.18752218 0.54879195]]
data[4, :, :] = 
[[0.77085415 0.49180735 1.43929596]
 [0.52368038 0.33410972 0.97778686]
 [0.46132233 0.29432509 0.86135538]
 [0.89490068 0.57094943 1.67090873]]

## Use reconstruct to compute part of a full tensor

In [15]:
X = generate_sample_ttensor() # Create a ttensor.
X.reconstruct(1,2) # Extract first front slice.

tensor of shape 5 x 4 x 1
data[0, :, :] = 
[[0.42425559]
 [0.28936173]
 [0.25467618]
 [0.49260853]]
data[1, :, :] = 
[[0.32061407]
 [0.22103447]
 [0.19406752]
 [0.37243706]]
data[2, :, :] = 
[[0.3070033 ]
 [0.21572321]
 [0.18859932]
 [0.35691607]]
data[3, :, :] = 
[[0.16187312]
 [0.10435162]
 [0.09305254]
 [0.18752218]]
data[4, :, :] = 
[[0.49180735]
 [0.33410972]
 [0.29432509]
 [0.57094943]]

## Use double to convert a ttensor to a (multidimensional) array

In [16]:
X = generate_sample_ttensor() # Create the ttensor.
X.double() # Converts to an array.

array([[[0.66497416, 0.42425559, 1.24160273],
        [0.45354281, 0.28936173, 0.84682989],
        [0.39917702, 0.25467618, 0.74532111],
        [0.77210991, 0.49260853, 1.44164064]],

       [[0.50252742, 0.32061407, 0.93829123],
        [0.34644731, 0.22103447, 0.64686714],
        [0.30417958, 0.19406752, 0.56794717],
        [0.58375429, 0.37243706, 1.08995351]],

       [[0.48119404, 0.3070033 , 0.89845873],
        [0.3381225 , 0.21572321, 0.63132351],
        [0.29560878, 0.18859932, 0.55194425],
        [0.55942684, 0.35691607, 1.04453065]],

       [[0.25371838, 0.16187312, 0.47372884],
        [0.16355973, 0.10435162, 0.30538962],
        [0.14584965, 0.09305254, 0.27232236],
        [0.29392047, 0.18752218, 0.54879195]],

       [[0.77085415, 0.49180735, 1.43929596],
        [0.52368038, 0.33410972, 0.97778686],
        [0.46132233, 0.29432509, 0.86135538],
        [0.89490068, 0.57094943, 1.67090873]]])

## Use ndims and size to get the size of a ttensor

In [17]:
X = generate_sample_ttensor() # Create the ttensor.
X.ndims # Number of dimensions.

3

In [18]:
X.shape # Row vector of the sizes.

(5, 4, 3)

In [19]:
X.shape[1] # Size of the 2nd mode.

4

## Subscripted reference for a ttensor

In [20]:
X = generate_sample_ttensor() # Create the ttensor.
X.core[0,0,0] # Access an element of the core.

0.5488135039273248

In [21]:
X.u[1] # Extract a matrix.

array([[0.79915856, 0.46147936],
       [0.78052918, 0.11827443],
       [0.63992102, 0.14335329],
       [0.94466892, 0.52184832]])

## **TODO**: We don't have a `__getitem__/subscripted reference ` for ttensor 

In [22]:
# X[1] # No __getitem__()

## Subscripted assignment for a ttensor

In [23]:
X = generate_sample_ttensor() # Create a ttensor.
X.core = ttb.tenones(X.core.shape) # Insert a new core.
X

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[1.]
	 [1.]]
	data[1, :, :] = 
	[[1.]
	 [1.]]
	data[2, :, :] = 
	[[1.]
	 [1.]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]

In [24]:
X.core[1,1,0] = 7 # Change a single element.
X

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[1.]
	 [1.]]
	data[1, :, :] = 
	[[1.]
	 [7.]]
	data[2, :, :] = 
	[[1.]
	 [1.]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]

## **TODO**: We don't have a `__getitem__/subscripted reference ` for ttensor 

In [25]:
# X[2][0:2,1] = [1,1] 

## Last index

## **TODO**: We don't have a `__getitem__/subscripted reference ` for ttensor 

In [26]:
# X[-1]

## Basic operations (uplus, uminus, mtimes, etc.) on a ttensor

In [27]:
X = generate_sample_ttensor() # Create ttensor.

### Addition

In [28]:
+X # Calls uplus.

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[0.5488135 ]
	 [0.71518937]]
	data[1, :, :] = 
	[[0.60276338]
	 [0.54488318]]
	data[2, :, :] = 
	[[0.4236548 ]
	 [0.64589411]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]

### Subtraction

In [29]:
-X # Calls uminus.

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[-0.5488135 ]
	 [-0.71518937]]
	data[1, :, :] = 
	[[-0.60276338]
	 [-0.54488318]]
	data[2, :, :] = 
	[[-0.4236548 ]
	 [-0.64589411]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]

### Multiplication

In [30]:
5*X # Calls mtimes.

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[2.74406752]
	 [3.57594683]]
	data[1, :, :] = 
	[[3.01381688]
	 [2.72441591]]
	data[2, :, :] = 
	[[2.118274  ]
	 [3.22947057]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]

## Use permute to reorder the modes of a ttensor

In [31]:
X = generate_sample_ttensor() # Create ttensor.
X.permute(np.array([2,1,0]))

Tensor of shape: (3, 4, 5)
	Core is a 	tensor of shape 1 x 2 x 3
	data[0, :, :] = 
	[[0.5488135  0.60276338 0.4236548 ]
	 [0.71518937 0.54488318 0.64589411]]
	U[0] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]

## Displaying a ttensor

The ttensor displays by displaying the core and each of the component matrices.

In [32]:
X = generate_sample_ttensor() # Create ttensor.
print(X)

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[0.5488135 ]
	 [0.71518937]]
	data[1, :, :] = 
	[[0.60276338]
	 [0.54488318]]
	data[2, :, :] = 
	[[0.4236548 ]
	 [0.64589411]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]



In [33]:
X # In the python interface

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[0.5488135 ]
	 [0.71518937]]
	data[1, :, :] = 
	[[0.60276338]
	 [0.54488318]]
	data[2, :, :] = 
	[[0.4236548 ]
	 [0.64589411]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]